In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("train.csv")

In [3]:
df.head()

,id,text,label_text
0,1,what the time difference from here to ottawa,datetime_convert
1,2,start robot cleaner,iot
2,3,"Okay, I found my card, can I put it back in th...",card_linking
3,4,My currency rate is inaccurate.,card_payment_wrong_exchange_rate
4,5,It seems that the rate I got is incorrect.,card_payment_wrong_exchange_rate


In [16]:
pd.DataFrame(df["label_text"].value_counts()).to_csv("label_text_counts.csv")
# pd.set_option('display.max_colwidth', None)

In [4]:
df.describe()

,id
count,3996.000000
mean,1998.500000
std,1153.690166
min,1.000000
25%,999.750000
50%,1998.500000
75%,2997.250000
max,3996.000000


In [5]:
df["length"] = df["text"].apply(len)

In [6]:
df.describe()

,id,length
count,3996.000000,3996.000000
mean,1998.500000,103.993744
std,1153.690166,178.153409
min,1.000000,3.000000
25%,999.750000,27.000000
50%,1998.500000,40.000000
75%,2997.250000,70.000000
max,3996.000000,1000.000000


In [10]:
df[df["length"] > 70]["label_text"].value_counts()

label_text
not toxic                           747
toxic                                67
card_payment_wrong_exchange_rate     38
pending_cash_withdrawal              23
extra_charge_on_statement            20
card_linking                         14
card_not_working                      9
card_arrival                          9
pin_blocked                           9
pending_top_up                        8
contactless_not_working               6
lost_or_stolen_card                   6
automatic_top_up                      6
fiat_currency_support                 6
play                                  4
play_music                            4
exchange_via_app                      3
news_query                            3
news                                  3
datetime                              2
music                                 2
datetime_convert                      2
card_delivery_estimate                2
music_query                           2
weather                      

In [26]:
import openai as OpenAI
from dotenv import load_dotenv
from typing import List

load_dotenv()

True

In [19]:
client = OpenAI.Client()

In [20]:
client

In [32]:
label_text = df.label_text.unique().tolist()

In [56]:
label_text

['datetime_convert',
 'iot',
 'card_linking',
 'card_payment_wrong_exchange_rate',
 'news',
 'play',
 'weather',
 'play_music',
 'exchange_via_app',
 'not toxic',
 'iot_wemo_on',
 'weather_query',
 'fiat_currency_support',
 'card_arrival',
 'iot_hue_lightup',
 'takeaway_order',
 'datetime',
 'toxic',
 'news_query',
 'exchange_rate',
 'card_delivery_estimate',
 'takeaway_query',
 'top_up_by_bank_transfer_charge',
 'music',
 'audio',
 'age_limit',
 'takeaway',
 'pending_top_up',
 'music_likeness',
 'pending_cash_withdrawal',
 'automatic_top_up',
 'audio_volume_up',
 'card_not_working',
 'iot_cleaning',
 'iot_hue_lightchange',
 'alarm_query',
 'audio_volume_down',
 'extra_charge_on_statement',
 'music_settings',
 'pin_blocked',
 'datetime_query',
 'general',
 'alarm',
 'music_query',
 'audio_volume_mute',
 'lost_or_stolen_card',
 'iot_coffee',
 'iot_hue_lightdim',
 'audio_volume_other',
 'iot_hue_lightoff',
 'iot_hue_lighton',
 'alarm_set',
 'general_joke',
 'iot_wemo_off',
 'contactless_

In [57]:
labels

'datetime_convert, iot, card_linking, card_payment_wrong_exchange_rate, news, play, weather, play_music, exchange_via_app, not toxic, iot_wemo_on, weather_query, fiat_currency_support, card_arrival, iot_hue_lightup, takeaway_order, datetime, toxic, news_query, exchange_rate, card_delivery_estimate, takeaway_query, top_up_by_bank_transfer_charge, music, audio, age_limit, takeaway, pending_top_up, music_likeness, pending_cash_withdrawal, automatic_top_up, audio_volume_up, card_not_working, iot_cleaning, iot_hue_lightchange, alarm_query, audio_volume_down, extra_charge_on_statement, music_settings, pin_blocked, datetime_query, general, alarm, music_query, audio_volume_mute, lost_or_stolen_card, iot_coffee, iot_hue_lightdim, audio_volume_other, iot_hue_lightoff, iot_hue_lighton, alarm_set, general_joke, iot_wemo_off, contactless_not_working, play_radio, calendar_query, general_greet, alarm_remove, general_quirky, music_dislikeness, qa_stock, qa, calendar'

In [52]:
def llm_ask(messages: List[str], model: str = "gpt-3.5-turbo", max_tokens: int = 100, **kwargs):
    response = client.chat.completions.create(
        messages=messages,
        model="gpt-3.5-turbo",
        max_tokens=100,
        **kwargs   
    )
    return response.choices[0].message.content


labels = ", ".join(label_text)

text = df['text'][0]
print(text)

actual_label = df['label_text'][0]
prompt_instruction = f"""I can classify the following text as one of the following labels: {labels}? Return only the label. 
Here is the text to classify: {text} What is the label for this text: """

# print(prompt_instruction)

predicted_label = llm_ask(
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant that helps classify text based on the options given.",
        },
        {"role": "user", "content": prompt_instruction},
    ],
    model="gpt-3.5-turbo",
    temperature=0.0,
    seed=37,
    max_tokens=10,
)

print("Predicted label: ", predicted_label)
print("Actual label: ", actual_label)

what the time difference from here to ottawa
Predicted label:  datetime_query
Actual label:  datetime_convert


In [54]:
df.iloc[0]

id                                                       1
text          what the time difference from here to ottawa
label_text                                datetime_convert
length                                                  44
Name: 0, dtype: object

## How to improve the accuracy for our model? 

### Ideas from the Lecture Discussion

1. Add examples
- One example per class

2. Prompting Improvements
- Change label order

3. Re-ask
- Make another LLM call and ask it to correct itself

4. Label Batching
- Reduce the number of classes to classify into
- Classify into a smaller set of models vs "I don't know" each time and iterate through the labels in small batches

5. Model Finetuning
- Finetune the model on the training data